# # 💡 Analisis Data Eksploratif: Deteksi Sinyal & Wawasan Fitur untuk Model Prediktif
#
# **Tujuan Notebook:**
# Notebook ini bertujuan untuk melakukan investigasi mendalam terhadap variabel-variabel (fitur) potensial yang akan digunakan dalam model prediksi kemacetan. Fokusnya adalah untuk memahami distribusi masing-masing fitur, hubungan antar fitur (multikolinearitas), dan yang terpenting, hubungan antara setiap fitur potensial dengan variabel target (misalnya, `avg_speed_kmh` atau kategori tingkat kemacetan).
#
# **Dataset yang Digunakan:** Data dari Gold Layer HDFS (data agregat yang sudah siap untuk rekayasa fitur atau pemodelan awal).
#
# **Tahapan Analisis:**
# 1. Setup Environment Spark & Pustaka
# 2. Pemuatan Data Agregat dari Gold Layer
# 3. Pemeriksaan Kualitas Data Fitur Potensial
# 4. Analisis Univariat Fitur Numerik (Distribusi, Outlier)
# 5. Analisis Univariat Fitur Kategorikal (Frekuensi Kategori)
# 6. Analisis Bivariat: Hubungan Fitur Numerik dengan Target (`avg_speed_kmh`)
# 7. Analisis Bivariat: Hubungan Fitur Kategorikal dengan Target (`avg_speed_kmh`)
# 8. Analisis Korelasi Antar Fitur Numerik (Deteksi Multikolinearitas)
# 9. (Opsional) Uji Signifikansi Fitur Sederhana (misalnya, ANOVA untuk kategorikal, korelasi untuk numerik)
# 10. Kesimpulan dan Rekomendasi Fitur untuk Tahap Rekayasa Fitur & Pemodelan


In [ ]:
# %% [code]
# 1. Setup Environment Spark & Pustaka
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, corr, count, when, desc, isnan
from pyspark.ml.stat import Correlation # Untuk matriks korelasi Spark
from pyspark.ml.feature import VectorAssembler # Jika ingin menghitung korelasi pada fitur vektor
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

spark = SparkSession.builder \
    .appName("EDAFeatureInsightsMedan") \
    .config("spark.sql.repl.eagerEval.enabled", True) \
    .getOrCreate()

print("SparkSession berhasil dibuat!")

# ## 2. Pemuatan Data Agregat dari Gold Layer
# Data ini sudah diagregasi per ruas jalan, jam, hari, dll., dan mencakup metrik trafik serta cuaca.


In [ ]:
gold_data_path = "/user/hive/warehouse/kemacetan_medan/gold/trafik_cuaca_agg/agregat_kemacetan.orc"
try:
    df_gold = spark.read.orc(gold_data_path)
    print(f"Data Gold berhasil dimuat. Jumlah baris: {df_gold.count()}")
    df_gold.printSchema()
    # Tampilkan beberapa kolom yang akan menjadi fitur dan target
    # Contoh kolom dari silver_to_gold.py: "road_name", "jam", "hari_minggu", "avg_speed_kmh", "vehicle_count", "avg_rainfall_mm", "avg_temperature_c"
    df_gold.show(5, truncate=False)
except Exception as e:
    print(f"Error memuat data Gold: {e}")

# Definisikan variabel target
target_col = "avg_speed_kmh"

# Definisikan fitur numerik dan kategorikal potensial (berdasarkan output gold_agg)
potential_numerical_features = ["jam", "hari_minggu", "vehicle_count", "avg_rainfall_mm", "avg_temperature_c"] # Tambahkan latitude, longitude jika per segmen
potential_categorical_features = ["road_name"]


# ## 3. Pemeriksaan Kualitas Data Fitur Potensial
# Memastikan tidak ada nilai null yang berlebihan atau tipe data yang salah pada fitur yang akan dianalisis.


In [ ]:
# Cek null untuk target dan fitur potensial
cols_to_check_null = [target_col] + potential_numerical_features + potential_categorical_features
print("Jumlah nilai null per kolom fitur potensial dan target:")
df_gold.select([count(when(col(c).isNull() | isnan(col(c)), c)).alias(c) for c in cols_to_check_null]).show()

# (Opsional) Lakukan imputasi sederhana jika ada null sedikit, atau drop baris jika banyak dan tidak bisa diimputasi dengan baik
# Untuk EDA, kita bisa filter baris dengan null pada target atau fitur kunci agar analisis tidak bias
df_gold_cleaned = df_gold.na.drop(subset=[target_col, "road_name"]) # Contoh
print(f"Jumlah baris setelah drop null pada target/fitur kunci: {df_gold_cleaned.count()}")


# ## 4. Analisis Univariat Fitur Numerik
# Memahami distribusi masing-masing fitur numerik.


In [ ]:
print("Statistik Deskriptif Fitur Numerik Potensial:")
df_gold_cleaned.select(potential_numerical_features).describe().show()

# Visualisasi distribusi (contoh untuk beberapa fitur)
# Ambil sampel jika data besar untuk plotting
numerical_sample_pd = df_gold_cleaned.select(potential_numerical_features).sample(False, 0.2).toPandas()

for num_feat in potential_numerical_features:
    if num_feat in numerical_sample_pd.columns:
        plt.figure(figsize=(8, 5))
        sns.histplot(numerical_sample_pd[num_feat], kde=True, bins=30)
        plt.title(f'Distribusi Fitur: {num_feat}')
        plt.xlabel(num_feat)
        plt.ylabel('Frekuensi')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.show()
        
        plt.figure(figsize=(8, 3))
        sns.boxplot(x=numerical_sample_pd[num_feat])
        plt.title(f'Boxplot Fitur: {num_feat} (Deteksi Outlier)')
        plt.show()

# **Interpretasi Distribusi Fitur Numerik:**
# * Fitur `jam` menunjukkan distribusi [seragam/bimodal/dll.], mengindikasikan [interpretasi].
# * Fitur `vehicle_count` cenderung [skewed ke kanan/kiri/simetris], dengan beberapa outlier [tinggi/rendah].
# * (Komentari setiap fitur numerik penting)

# ## 5. Analisis Univariat Fitur Kategorikal
# Memahami frekuensi kategori pada fitur kategorikal.


In [ ]:
for cat_feat in potential_categorical_features:
    print(f"\nFrekuensi Kategori untuk Fitur: {cat_feat}")
    df_gold_cleaned.groupBy(cat_feat).count().orderBy(desc("count")).show(truncate=False)
    
    # Visualisasi (misalnya, top N kategori)
    top_n_categories = 15
    category_counts_pd = df_gold_cleaned.groupBy(cat_feat) \
        .count() \
        .orderBy(desc("count")) \
        .limit(top_n_categories) \
        .toPandas()

    if not category_counts_pd.empty:
        plt.figure(figsize=(12, 7))
        sns.barplot(data=category_counts_pd, y=cat_feat, x="count", palette="Spectral")
        plt.title(f'Top {top_n_categories} Kategori Teratas untuk Fitur: {cat_feat}')
        plt.xlabel('Jumlah Observasi')
        plt.ylabel(cat_feat)
        plt.tight_layout()
        plt.show()


# **Interpretasi Distribusi Fitur Kategorikal:**
# * Fitur `road_name` memiliki [jumlah] kategori unik. Beberapa ruas jalan muncul jauh lebih sering daripada yang lain, menunjukkan variasi kepadatan data per lokasi.
# * Adanya kategori "Unknown" atau yang sangat jarang mungkin memerlukan penanganan khusus (misalnya, penggabungan atau penghapusan).


# ## 6. Analisis Bivariat: Hubungan Fitur Numerik dengan Target (`avg_speed_kmh`)
# Melihat bagaimana nilai target berubah seiring perubahan fitur numerik.


In [ ]:
# Scatter plot untuk setiap fitur numerik vs target (ambil sampel)
bivariate_sample_pd = df_gold_cleaned.select([target_col] + potential_numerical_features).sample(False, 0.1).toPandas()

for num_feat in potential_numerical_features:
    if num_feat in bivariate_sample_pd.columns:
        plt.figure(figsize=(8, 6))
        sns.scatterplot(data=bivariate_sample_pd, x=num_feat, y=target_col, alpha=0.5)
        # Opsional: tambahkan garis regresi
        # sns.regplot(data=bivariate_sample_pd, x=num_feat, y=target_col, scatter_kws={'alpha':0.3}, line_kws={'color':'red'})
        plt.title(f'Hubungan Antara {num_feat} dan {target_col}')
        plt.xlabel(num_feat)
        plt.ylabel(target_col)
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.show()

# **Interpretasi Hubungan Fitur Numerik dengan Target:**
# * `jam` vs `avg_speed_kmh`: Menunjukkan pola non-linear yang jelas, dengan kecepatan menurun pada jam sibuk (seperti yang terlihat di EDA trafik). Ini menguatkan `jam` sebagai fitur penting.
# * `vehicle_count` vs `avg_speed_kmh`: Cenderung menunjukkan korelasi negatif; semakin banyak kendaraan, kecepatan rata-rata menurun.
# * `avg_rainfall_mm` vs `avg_speed_kmh`: Tampak ada tren penurunan kecepatan saat curah hujan meningkat.
# * (Komentari setiap plot)


# ## 7. Analisis Bivariat: Hubungan Fitur Kategorikal dengan Target (`avg_speed_kmh`)
# Melihat bagaimana nilai target bervariasi antar kategori fitur.


In [ ]:
for cat_feat in potential_categorical_features:
    # Ambil top N kategori berdasarkan jumlah observasi untuk visualisasi yang lebih bersih
    top_n_plot = 15
    top_categories_df = df_gold_cleaned.groupBy(cat_feat).count().orderBy(desc("count")).limit(top_n_plot)
    top_categories_list = [row[cat_feat] for row in top_categories_df.collect()]

    df_cat_target_pd = df_gold_cleaned.filter(col(cat_feat).isin(top_categories_list)) \
        .select(cat_feat, target_col) \
        .toPandas() # Hati-hati jika hasilnya masih besar

    if not df_cat_target_pd.empty:
        plt.figure(figsize=(14, 8))
        sns.boxplot(data=df_cat_target_pd, y=cat_feat, x=target_col, palette="Paired", order=df_cat_target_pd.groupby(cat_feat)[target_col].median().sort_values().index)
        plt.title(f'Distribusi {target_col} per Kategori {cat_feat} (Top {top_n_plot})')
        plt.xlabel(target_col)
        plt.ylabel(cat_feat)
        plt.tight_layout()
        plt.show()

# **Interpretasi Hubungan Fitur Kategorikal dengan Target:**
# * `road_name` vs `avg_speed_kmh`: Boxplot menunjukkan variasi yang signifikan dalam kecepatan rata-rata antar ruas jalan. Beberapa ruas jalan secara konsisten memiliki kecepatan lebih rendah (potensi macet). Ini menegaskan `road_name` sebagai fitur prediktif yang kuat.


# ## 8. Analisis Korelasi Antar Fitur Numerik (Deteksi Multikolinearitas)
# Memeriksa apakah ada fitur numerik yang sangat berkorelasi satu sama lain, yang bisa menjadi masalah untuk beberapa model.


In [ ]:
# Menggunakan metode korelasi dari Spark DataFrame (lebih efisien untuk data besar)
# Buat vektor fitur dari kolom numerik
assembler_corr = VectorAssembler(inputCols=potential_numerical_features, outputCol="corr_features_vec", handleInvalid="skip")
df_corr_vec = assembler_corr.transform(df_gold_cleaned).select("corr_features_vec")

# Hitung matriks korelasi Pearson
try:
    corr_matrix_spark = Correlation.corr(df_corr_vec, "corr_features_vec", "pearson").head()
    if corr_matrix_spark:
        corr_matrix_pd_from_spark = pd.DataFrame(corr_matrix_spark[0].toArray(), columns=potential_numerical_features, index=potential_numerical_features)
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(corr_matrix_pd_from_spark, annot=True, cmap='coolwarm', fmt=".2f", vmin=-1, vmax=1)
        plt.title('Matriks Korelasi Antar Fitur Numerik Potensial')
        plt.show()
    else:
        print("Tidak dapat menghitung matriks korelasi (mungkin karena data terlalu sedikit setelah filter).")
except Exception as e:
    print(f"Error menghitung matriks korelasi Spark: {e}")
    print("Alternatif: Menggunakan Pandas pada sampel data (kurang akurat untuk data besar).")
    # Fallback ke Pandas jika ada error atau data sampel
    # corr_matrix_pd = numerical_sample_pd.corr()
    # plt.figure(figsize=(10, 8))
    # sns.heatmap(corr_matrix_pd, annot=True, cmap='coolwarm', fmt=".2f")
    # plt.title('Matriks Korelasi Antar Fitur Numerik Potensial (Sampel Pandas)')
    # plt.show()

# **Interpretasi Matriks Korelasi Fitur:**
# * Perhatikan pasangan fitur dengan koefisien korelasi tinggi (misalnya, > 0.7 atau < -0.7).
# * Jika `avg_rainfall_mm` dan `humidity_percent` sangat berkorelasi, mungkin salah satunya bisa dipertimbangkan untuk dihilangkan untuk menghindari multikolinearitas.
# * (Komentari temuan korelasi penting)


# ## 9. (Opsional) Uji Signifikansi Fitur Sederhana
# (Bagian ini bisa lebih kompleks, tergantung kedalaman yang diinginkan. Untuk EDA, korelasi dan visualisasi seringkali cukup.)
# - **Numerik vs Numerik (Target)**: Koefisien korelasi Pearson sudah dihitung.
# - **Kategorikal vs Numerik (Target)**: ANOVA (Analysis of Variance) dapat digunakan untuk menguji apakah rata-rata target berbeda signifikan antar kategori. Ini memerlukan transformasi data atau penggunaan pustaka statistik.
#   ```python
#   # Contoh konseptual (memerlukan scipy atau statsmodels, mungkin lebih baik di Pandas)
#   # from scipy.stats import f_oneway
#   # grouped_data = [df_cat_target_pd[df_cat_target_pd[cat_feat] == category][target_col] for category in df_cat_target_pd[cat_feat].unique()]
#   # f_stat, p_value = f_oneway(*grouped_data)
#   # print(f"ANOVA P-value untuk {cat_feat} vs {target_col}: {p_value}")
#   ```

# ## 10. Kesimpulan dan Rekomendasi Fitur untuk Tahap Rekayasa Fitur & Pemodelan
# Berdasarkan eksplorasi mendalam terhadap fitur-fitur potensial:
# 1.  **Fitur Numerik yang Menjanjikan**:
#     * `jam` dan `hari_minggu`: Menunjukkan pola temporal yang kuat terhadap kecepatan.
#     * `vehicle_count`: Berkorelasi negatif dengan kecepatan, sangat intuitif.
#     * `avg_rainfall_mm` dan `visibility_km` (dari EDA cuaca): Menunjukkan dampak pada kecepatan.
#     * `avg_temperature_c`: Dampaknya mungkin perlu dipertimbangkan hati-hati atau dikombinasikan.
# 2.  **Fitur Kategorikal yang Menjanjikan**:
#     * `road_name`: Sangat penting, menunjukkan variasi kecepatan yang signifikan antar lokasi. Perlu di-encode dengan benar (misalnya, `StringIndexer` atau target encoding jika kategorinya banyak).
# 3.  **Potensi Multikolinearitas**: [Sebutkan jika ada fitur yang sangat berkorelasi dan perlu perhatian, misal: "Perlu diperhatikan korelasi antara X dan Y"].
# 4.  **Rekomendasi untuk Rekayasa Fitur**:
#     * Fokus pada encoding `road_name` yang efektif.
#     * Pertimbangkan pembuatan fitur interaksi (misalnya, `jam` * `road_name_index`, atau `jam` * `avg_rainfall_mm`).
#     * Normalisasi/scaling fitur numerik mungkin diperlukan tergantung algoritma model.
#     * Pembuatan fitur lag (kecepatan pada jam sebelumnya) sangat direkomendasikan.
#
# Insight dari EDA ini akan menjadi **panduan utama** dalam skrip `prepare_features_for_modeling.py` untuk memastikan bahwa fitur yang dimasukkan ke dalam model adalah yang paling informatif dan relevan, sehingga memaksimalkan potensi akurasi prediksi kemacetan.


In [ ]:
spark.stop()